---
# --- Day 15: Beacon Exclusion Zone ---
---

In [19]:
import numpy as np
import re
from math import sqrt, floor
from tqdm.notebook import tqdm
from itertools import product

## Load data

In [2]:
full_puzzle_data = True

In [3]:
file_suffix = "" if full_puzzle_data else "_test"
with open(f"data/day15_input{file_suffix}.txt", "r") as f:
    data = f.read().splitlines()

In [4]:
positions = []
for r in data:
    m = re.findall("Sensor at x=([-]?[0-9]+), y=([-]?[0-9]+): closest beacon is at x=([-]?[0-9]+), y=([-]?[0-9]+)", r)[0]
    positions += [[int(d) for d in m]]

In [5]:
target_row = 2000000 if full_puzzle_data else 10
max_value = 4000000 if full_puzzle_data else 20

## --- Part One ---

In [ ]:
sensors = {}
beacons = {}
empties = {}

for p in positions:
    ys, xs, yb, xb = p
    sensors[xs] = sensors.get(xs, set()).union(set([ys]))
    beacons[xb] = beacons.get(xb, set()).union(set([yb]))
    manh = abs(xs-xb) + abs(ys-yb)
    for i in range(-manh, manh+1):
        if xs+i == target_row:
            js = list(range(-manh, manh+1))
            sy = set([ys+j for j in range(-manh, manh+1) if abs(i)+abs(j) <= manh])
            empties[xs+i] = empties.get(xs+i, set()).union(sy)

In [ ]:
num_nobeacon = len(empties[target_row].difference(beacons.get(target_row, set())))
print(f"Number of positions that cannot contain a beacon: {num_nobeacon}.")

## --- Part Two ---

### 1. Approach with axis rotation (it doesn't work with high dimensions)

In [ ]:
def rotate_axis(p):
    return (p[0]+p[1], p[1]-p[0])

def rotate_back(p):
    return (int((p[0]-p[1])/2), int((p[0]+p[1])/2))

In [ ]:
free = {}
for i in range(max_value+1):
    pos = set(range(-i, i+1, 2))
    free[i] = pos
    if i != max_value:
        free[2*max_value - i] = pos

In [ ]:
xset = set(range(0, 2*max_value+1))


for k, p in enumerate(positions):
    ys, xs, yb, xb = p    
    
    manh = floor((abs(xs-xb) + abs(ys-yb)))
    newx, newy = rotate_axis((xs, ys))
    for i in xset.intersection(set(range(newx-manh, newx+manh+1))):
        free[i] = free[i].difference(set(range(newy-manh, newy+manh+1)))

In [ ]:
for k in free.keys():
    if len(free[k]) > 1:
        print("Many possible combinations")
    if len(free[k]) == 1:
        print(rotate_back((k, free[k].pop())))

### 2. Iterative approach

In [6]:
def is_block_covered(b_coord, sensors, mdist):
    i = 0
    covered = False
    p = np.array([[b_coord[0], b_coord[2]], [b_coord[0], b_coord[3]], [b_coord[1], b_coord[2]], [b_coord[1], b_coord[3]]])
    while not covered and (i < len(sensors)):
        s = sensors[i].reshape(1,2)
        covered = np.all(np.sum(abs(p - np.ones((4,1)).dot(s)), axis=1) <= mdist[i])
        i += 1
    return covered

In [7]:
def find_uncovered_blocks(search_area_coords, n, sensors, mdist):
    unc_coords = []
    for search_area in tqdm(search_area_coords, total=len(search_area_coords)):
        xs = [int(np.round(x)) for x in np.linspace(search_area[0], search_area[1], n+1)]
        ys = [int(np.round(y)) for y in np.linspace(search_area[2], search_area[3], n+1)]
        for i in range(len(xs)-1):
            deltai = 1 if i < len(xs) - 2 else 0
            for j in range(len(ys)-1):
                deltaj = 1 if j < len(ys) - 2 else 0
                block = (xs[i], xs[i+1]-deltai, ys[j], ys[j+1]-deltaj)
                covered = is_block_covered(block, sensors, mdist)
                if not covered:
                    unc_coords.append(block)
    print(f"Non covered blocks: {len(unc_coords)}.")
    bsize = unc_coords[0][1] - unc_coords[0][0] +1 
    print(f"Block size: {bsize}x{bsize}.")
    return unc_coords

In [8]:
sensors = []
mdist = []
for k, p in enumerate(positions):
    ys, xs, yb, xb = p    
    
    mdist.append(abs(xs-xb) + abs(ys-yb))
    sensors.append([xs, ys])
mdist = np.array(mdist)
sensors = np.array(sensors)

In [9]:
search_areas =  [(0, max_value, 0, max_value)]
unc_coords = find_uncovered_blocks(search_areas, 500, sensors, mdist)

  0%|          | 0/1 [00:00<?, ?it/s]

Non covered blocks: 120.
Block size: 8000x8000.


In [10]:
unc_coords = find_uncovered_blocks(unc_coords, 100, sensors, mdist)

  0%|          | 0/120 [00:00<?, ?it/s]

Non covered blocks: 10895.
Block size: 81x81.


In [11]:
unc_coords = find_uncovered_blocks(unc_coords, 20, sensors, mdist)

  0%|          | 0/10895 [00:00<?, ?it/s]

Non covered blocks: 130228.
Block size: 4x4.


#### check on all remaining points

In [12]:
rem = []
for u in unc_coords:
    rem += list(product(range(u[0], u[1]+1), range(u[2], u[3]+1)))
print(f"{len(rem)} points to check.")

2103362 points to check.


In [13]:
def check_point(p, sensors, mdist):
    p = np.array(p).reshape(1, 2)
    covered = np.any(np.sum(abs(sensors - np.ones((32,1)).dot(p)), axis=1) <= mdist)
    return covered

In [16]:
list_unc = []
for p in tqdm(rem, total=len(rem)):
    covered = check_point(p, sensors, mdist)
    if not covered:
        list_unc.append(p)

  0%|          | 0/2103362 [00:00<?, ?it/s]

In [18]:
freq = list_unc[0][1]*4000000+list_unc[0][0]
print(f"The tuning frequency of the distress signal is {freq}.")

The tuning frequency of the distress signal is 11840879211051.
